In [1]:
from langchain_experimental.graph_transformers.diffbot import DiffbotGraphTransformer
from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()

True

In [2]:
diffbot_nlp = DiffbotGraphTransformer(diffbot_api_key=os.getenv("DIFFBOT_API_KEY"))

In [3]:
from langchain_community.document_loaders import WikipediaLoader

query = "Warren Buffett"
raw_documents = WikipediaLoader(query=query).load()
graph_documents = diffbot_nlp.convert_to_graph_documents(raw_documents)

In [4]:
graph_documents[0]

GraphDocument(nodes=[Node(id='http://www.wikidata.org/entity/Q47213', type='Person', properties={'name': 'Warren Buffett', 'positionHeld': 'CEO', 'age': '7', 'dateOfBirth': '1930-08-30'}), Node(id='http://www.wikidata.org/entity/Q43199', type='Location', properties={'name': 'Omaha'}), Node(id='http://www.wikidata.org/entity/Q723488', type='Person', properties={'name': 'Howard Homan Buffett', 'positionHeld': 'businessman'}), Node(id='http://www.wikidata.org/entity/Q451695', type='Person', properties={'name': 'Lesley Stahl'}), Node(id='Buffett Partnership Ltd.', type='Organization', properties={'name': 'Buffett Partnership Ltd.', 'foundingDate': '1956'}), Node(id='Leila', type='Person', properties={'name': 'Leila', 'numberOfChildren': '3'}), Node(id='http://www.wikidata.org/entity/Q907481', type='Organization', properties={'name': 'Columbia Business School'}), Node(id='http://www.wikidata.org/entity/Q203807', type='Organization', properties={'name': 'The Giving Pledge', 'foundingDate': '

In [5]:
from langchain_neo4j import Neo4jGraph

url = "bolt://localhost:7687"
username = "neo4j"
password = "password"

graph = Neo4jGraph(url=url, username=username, password=password)

In [6]:
graph.add_graph_documents(graph_documents)

In [7]:
graph.refresh_schema()

# Query the graph

In [16]:
from langchain_neo4j import GraphCypherQAChain
from langchain_aws import ChatBedrock
from config import env

chat_bedrock_model = ChatBedrock(
    temperature=0,
    model="us.anthropic.claude-3-5-sonnet-20240620-v1:0",
    region=env.AWS_REGION,
    aws_access_key_id=env.AWS_ACCESS_KEY_ID,
    aws_secret_access_key=env.AWS_SECRET_ACCESS_KEY,
)

chain = GraphCypherQAChain.from_llm(
    cypher_llm=chat_bedrock_model,
    qa_llm=chat_bedrock_model,
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
)

In [17]:
chain.run("Which university did Warren Buffett attend?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Warren Buffett"})-[:EDUCATED_AT]->(o:Organization)
WHERE o.name CONTAINS "University"
RETURN o.name AS University
Full Context:
[{'University': 'University of Pennsylvania'}, {'University': 'University of Nebraska-Lincoln'}]

> Finished chain.


'Warren Buffett attended the University of Nebraska-Lincoln and the University of Pennsylvania.'